# Домашнее задание № 1 (Apache Spark SQL)

**Цель**: Решить 5 задач на SQL с использованием базовых таблиц.

**Инструкции**:
0. Прочитать условия ДЗ
1. Выполнить задание на SQL;
2. Ознакомиться с документацией по SQL: [Ссылка](https://spark.apache.org/docs/latest/sql-ref.html);
3. Создать ноутбук с решениями задач (создайте копию на основе этого шаблона);
4. Каждый запрос должен содержать обязательные комментарии с объяснением логического смысла решения;
5. Скачать ноутбук в формате .ipynb и отправить на адрес электронной почты: ilya+hse@aniskovets.com.

**Оценка и дедлайн**:
- Максимальная оценка - 5 баллов;
- Дедлайн: 24.05.2023 23:59;
- Оценки будут опубликованы после сдачи **всех двух** домашних заданий.

**Таблицы**:
Для выполнения задания предполагается использование следующих таблиц, загруженных в систему в формате CSV с типами полей STRING:

- title_basics_csv
- title_principals_csv
- title_crew_csv
- title_episode_csv
- title_ratings_csv
- title_akas_csv
- name_basics_csv

**Преобразование типов**:
Для преобразования STRING в другой тип используйте конструкцию: 
```sql
CAST (column_name AS TYPE) AS column_name
```

**Пример**
```sql
CREATE TABLE test_table USING PARQUET AS 
  SELECT 
    CAST(averageRating AS decimal(2,1))  AS averageRating 
  FROM test_table_csv 
```

**Все подготовительные этапы по преобразованию типов нужно делать в разделе Initialization. После чего, рекомендуется использовать в запросах таблицы в формате PARQUET**

Перед этим не мешает проверить, что значения влезают в размерности этого типа
Список типов SQL: https://spark.apache.org/docs/latest/sql-ref-datatypes.html 

Домашнее задание необходимо сделать в виде ноутбука. Ноутбук должен запускаться без ошибок.

Во всех задачах ответом должно быть только ОДНО значение (число или строка).

Неправильный ответ - 0 баллов за задачу.
Правильный ответ на все тесты 1 балл за задачу, иначе пропорционально количеству пройденных тестов (количество тестов и какие именно заранее неизвестны).

**ВНИМАНИЕ! К каждому запросу необходимо писать комментарии. Комментарии должны обьяснять логический смысл решения. Отсутствие комментария к решению задачи штраф - оценка за задачу = оценка / 2!**

Всего 5 задач и 5 баллов

**В режиме проверки SQL запросы будут запускаться с любыми параметрами, отличными от тех, что вы выбрали.**

Скрипты не должны зависеть от выбранного вами параметра, а также от регистра строки. Все параметры должны быть изменяемыми (смотрите, пример в Вопросе № 0)

**Тестирование будет проводится с любым значением параметров, в том числе с отсутвующим в датасете, запрос должен выводить правильный результат!**

SQL запрос в ответе всегда должен быть один и параметризирован. Если необходимо пользуйтесь конструкцией WITH name AS () -  пример ниже

После завершения работы нажимайте в меню File/Download/Download ipnb, скачивайте файл и присылайте почтой на ilya+hse@aniskovets.com

Если заметили опечатки или появились вопросы пишите в телеграм: @aigmx



In [1]:
!pip install pyspark===3.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=487e71a03219af26db72d975f7b04daebbc58bdef4405735c70499447268d941
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
!curl -O https://mars.ru77.ru/data/title.basics.tsv.gz
!curl -O https://mars.ru77.ru/data/title.crew.tsv.gz
!curl -O https://mars.ru77.ru/data/title.episode.tsv.gz
!curl -O https://mars.ru77.ru/data/title.principals.tsv.gz
!curl -O https://mars.ru77.ru/data/title.ratings.tsv.gz
!curl -O https://mars.ru77.ru/data/title.akas.tsv.gz
!curl -O https://mars.ru77.ru/data/name.basics.tsv.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  132M  100  132M    0     0  10.0M      0  0:00:13  0:00:13 --:--:-- 15.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 51.7M  100 51.7M    0     0  7270k      0  0:00:07  0:00:07 --:--:-- 12.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29.6M  100 29.6M    0     0  4755k      0  0:00:06  0:00:06 --:--:-- 6692k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  342M  100  342M    0     0  10.1M      0  0:00:33  0:00:33 --:--:-- 14.7M
  % Total    % Received % Xferd  Average Speed   Tim

In [3]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf, SparkContext


spark = SparkSession.builder.master("local[2]").config("spark.driver.memory", "8g").appName("vse").enableHiveSupport().getOrCreate()
sql = spark.sql

In [4]:
title_basics_csv = spark.read.csv("title.basics.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_basics_csv.createOrReplaceTempView("title_basics_csv")


title_principals_csv = spark.read.csv("title.principals.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_principals_csv.createOrReplaceTempView("title_principals_csv")


title_crew_csv = spark.read.csv("title.crew.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_crew_csv.createOrReplaceTempView("title_crew_csv")


title_episode_csv = spark.read.csv("title.episode.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_episode_csv.createOrReplaceTempView("title_episode_csv")


title_ratings_csv = spark.read.csv("title.ratings.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_ratings_csv.createOrReplaceTempView("title_ratings_csv")


title_akas_csv = spark.read.csv("title.akas.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_akas_csv.createOrReplaceTempView("title_akas_csv")


name_basics_csv = spark.read.csv("name.basics.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
name_basics_csv.createOrReplaceTempView("name_basics_csv")

In [5]:
# Initialization
_ = sql("""DROP TABLE IF EXISTS title_basics""").collect()
_ = sql("""CREATE TABLE title_basics STORED AS PARQUET SELECT * FROM title_basics_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS title_principals""").collect()
_ = sql("""CREATE TABLE title_principals STORED AS PARQUET SELECT * FROM title_principals_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS title_crew""").collect()
_ = sql("""CREATE TABLE title_crew STORED AS PARQUET SELECT * FROM title_crew_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS title_episode""").collect()
_ = sql("""CREATE TABLE title_episode STORED AS PARQUET SELECT * FROM title_episode_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS title_ratings""").collect()
_ = sql("""CREATE TABLE title_ratings STORED AS PARQUET SELECT * FROM title_ratings_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS title_akas""").collect()
_ = sql("""CREATE TABLE title_akas STORED AS PARQUET SELECT * FROM title_akas_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS name_basics""").collect()
_ = sql("""CREATE TABLE name_basics STORED AS PARQUET SELECT * FROM name_basics_csv""").collect()

In [6]:
sql("DESCRIBE FORMATTED title_basics").show(50, truncate=False)
sql("DESCRIBE FORMATTED title_principals").show(50, truncate=False)
sql("DESCRIBE FORMATTED title_crew").show(50, truncate=False) 
sql("DESCRIBE FORMATTED title_episode").show(50, truncate=False)
sql("DESCRIBE FORMATTED title_ratings").show(50, truncate=False)
sql("DESCRIBE FORMATTED title_akas").show(50, truncate=False)
sql("DESCRIBE FORMATTED name_basics").show(50, truncate=False)

+----------------------------+--------------------------------------------------------------+-------+
|col_name                    |data_type                                                     |comment|
+----------------------------+--------------------------------------------------------------+-------+
|tconst                      |string                                                        |null   |
|titleType                   |string                                                        |null   |
|primaryTitle                |string                                                        |null   |
|originalTitle               |string                                                        |null   |
|isAdult                     |string                                                        |null   |
|startYear                   |string                                                        |null   |
|endYear                     |string                                              

Вопрос №0 (не оценивается, показан, как пример). Сколько произведений имеют средний рейтинг выше заданного {threshold}?

In [ ]:
query_0 = """
/*
  Чтобы посчитать количество произведений находим все рейтинги больше, чем заданный порог.
  Так как надо найти все произведения, а не только фильмы, нам не нужно делать фильтр по типу произведения titleType
*/
SELECT 
  COUNT(tconst) as cnt_above_threshold
FROM  title_ratings
WHERE averageRating > {threshold};
"""
sql(query_0, threshold=5.0).show(truncate=False)

+-------------------+
|cnt_above_threshold|
+-------------------+
|1029097            |
+-------------------+



Вопрос №1. Найдите среднюю продолжительность (title_basics:runtimeMinutes) фильмов (title_basics:titleType=movie) определенного жанра {genre}. 

Среднюю продолжительность надо рассчитать только для тех фильмов, где title_basics:runtimeMinutes не равен NULL. 

In [87]:
query_1 = """
SELECT /*задаем набор данных из таблицы */
  AVG(runtimeMinutes) as answer /* берем среднее от продолжительности фильма в минутах */
FROM title_basics /*  из таблицы title_basics */
WHERE (titleType = 'movie' AND genres ILIKE {genre}); /*для условия что произведение - фильм и жанр произведения равен заданному жанру (комедия), используем ILIKE, чтобы не зависеть от задания параметра genre*/
"""
sql(query_1, genre='%comedY%').show(truncate=False)

+----------------+
|answer          |
+----------------+
|92.5809266708541|
+----------------+



Вопрос №2. Сколько фильмов (title_basics:titleType=movie) определенного жанра (title_basics:genres) {genre} было выпущено в определенный год (title_basics:startYear) {year}?


In [88]:
query_2 = """
SELECT /*задаем набор данных из таблицы */
  COUNT(*) as answer /*количество строк*/
FROM title_basics /* из таблицы title_basics */
WHERE (titleType = 'movie' AND  genres ILIKE {genre} AND startYear = {year});  /*для условия что произведение - фильм, жанр произведения равен заданному жанру (комедия) и год выпуска равен заданному (year)*/
"""
sql(query_2, genre='%Comedy%', year=2022).show(truncate=False)

+------+
|answer|
+------+
|203   |
+------+



Вопрос №3. Сколько фильмов определенного жанра (title_basics:genres) {genre}, в которых снялся конкретный актер (name_basics:primaryName) {actor}, имеют средний рейтинг (title_ratings:averageRating) выше заданного порога {threshold}?







In [90]:
query_3 = """
SELECT /*задаем набор данных из таблицы */
  COUNT(*) as answer  /*количество строк*/
FROM title_basics /* из таблицы title_basics */
JOIN title_ratings ON title_basics.tconst = title_ratings.tconst /* пересеченной с таблицей title_ratings по ключу tconst для получения рейтинга фильма */
JOIN title_principals ON title_basics.tconst = title_principals.tconst /* пересеченной с таблицей title_principals по ключу tconst для получения category и связки с таблицей с именами */
JOIN name_basics ON title_principals.nconst = name_basics.nconst /* пересеченной с таблицей name_basics по ключу nconst(ключи имен) для получения для сравнения с именем актера */
WHERE (title_basics.titleType = 'movie' AND title_ratings.averageRating > {threshold} AND title_basics.genres ILIKE {genre} AND name_basics.primaryName ILIKE {actor} AND title_principals.category = 'actor');
/* для условия что произведение - фильм, рейтинг произведения больше заданного (threshold), жанр произведения равен заданному жанру (комедия), имя персоны совпадает с заданным (actor), и что персона является актером*/
"""
sql(query_3, genre='%Comedy%', actor='James Cagney', threshold=5.3).show(truncate=False)

+------+
|answer|
+------+
|20    |
+------+



Вопрос №4. Сколько телесериалов (title_basics:titleType=tvSeries) в определенном жанре (title_basics:genres) {genre} имеют более определенного количества (title_episode:seasonNumber > {seasons})?


In [91]:
query_4 = """
SELECT /*задаем набор данных из таблицы */
  COUNT(DISTINCT(title_episode.parentTconst)) as answer /*количество уникальных по ключу произведения строк*/
FROM title_basics /* из таблицы title_basics */
JOIN title_episode ON title_basics.tconst = title_episode.parentTconst /* пересеченной с таблицей title_episode по ключу tconst в title_basics = parentTconst в title_episode (tconst в title_episode - ключ эпизода, а parent - сериала) для получения количества сезонов */
WHERE (title_basics.titleType = 'tvSeries' AND title_basics.genres ILIKE {genre} AND title_episode.seasonNumber > {seasons});
 /* для условия что произведение - телесериал, жанр произведения равен заданному жанру (комедия) и количество сезонов больше заданного (seasons)*/
"""
sql(query_4, genre='%Comedy%', seasons=7).show(truncate=False)

+------+
|answer|
+------+
|995   |
+------+



Вопрос № 5. Сколько фильмов (title_basics:titleType=movie)  создал конкретный сценарист (title_crew:writers, name_basics:primaryName) {writer} в определенном жанре (title_basics:genres) {genre}?

In [96]:
query_5  = """
SELECT /*задаем набор данных из таблицы */
  COUNT(*) as answer /*количество строк*/
FROM title_crew /* из таблицы title_crew */
JOIN name_basics ON CONTAINS(title_crew.writers, name_basics.nconst) /* пересеченной с таблицей name_basics по содержанию персоны в списке сценаристов для получения сценаристов фильмов*/
JOIN title_basics ON title_basics.tconst = title_crew.tconst /* пересеченной с таблицей title_basics по ключу tconst для получения жанров и типов произведения*/
WHERE (title_basics.titleType = 'movie' AND  title_basics.genres ILIKE {genre} AND name_basics.primaryName ILIKE {writer});
 /* для условия что произведение - фильм, жанр произведения равен заданному жанру (комедия) и имя персоны (которую в первом JOINе мы уже определили как сценариста) совпадает с заданным (writer)*/
"""
sql(query_5, genre='%Comedy%', writer='James Cagney').show(truncate=False)

+------+
|answer|
+------+
|0     |
+------+



In [85]:
/# PLEASE DO NOT REMOVE THIS CELL

# Generate answers
for n in range(1, 6):
  print(f"-- answer {n}\n " + globals()[f"query_{n}"] + ";\n")

-- answer 1
 
SELECT 
  AVG(runtimeMinutes) as answer
FROM title_basics
WHERE (titleType = 'movie' AND genres ILIKE {genre});
;

-- answer 2
 
SELECT 
  COUNT(titleType = 'movie') as answer
FROM title_basics
WHERE (titleType = 'movie' AND  genres ILIKE {genre} AND startYear = {year});
;

-- answer 3
 
SELECT COUNT(title_basics.tconst) as answer
FROM title_basics
JOIN title_ratings ON title_basics.tconst = title_ratings.tconst
JOIN title_principals ON title_basics.tconst = title_principals.tconst
JOIN name_basics ON title_principals.nconst = name_basics.nconst
WHERE (title_basics.titleType = 'movie' AND title_ratings.averageRating > {threshold} AND title_basics.genres ILIKE {genre} AND name_basics.primaryName ILIKE {actor} AND title_principals.category = 'actor');
;

-- answer 4
 
SELECT COUNT(DISTINCT(title_episode.parentTconst)) as answer
FROM title_basics
JOIN title_episode ON title_basics.tconst = title_episode.parentTconst
WHERE (title_basics.titleType = 'tvSeries' AND title_basics